# Polígonos de quadras com tipos de uso

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np

## Carregando e pre-processando os dados de IPTU

In [2]:
# Padrões de nomenclatura de usos

usos = {
    "apartamento em condomínio":"apartamento",
    "cortiço (habitação coletiva subnormal)":"cortiço",
    "escritório/consultório em condomínio (unidade autônoma)":"escritório ou consultório",
    "loja em edifício em condomínio (unidade autônoma)":"loja em edifício em condomínio",
    "residência coletiva, exclusive cortiço (mais de uma residência no lote)":"residência coletiva (mais de uma residência no lote), exclusive cortiço",
    "posto de serviço (combustíveis)":"posto de serviço",
    "estacionamento e garagem, não em condomínio":"garagem (exclusive em prédio em condomínio)",
    "garagem (unidade autônoma) de prédio de garagens":"garagem, em prédio de garagens",
    "garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto":"garagem em edifício de escritórios, consultórios ou misto",
    "garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial":"garagem em edifício de uso exclusivamente residencial",
    "outras edificações de uso coletivo, com utilização múltipla":"outras edificações do tipo (uso coletivo), com utilização múltipla",
    "outras edificações de uso comercial, com utilização múltipla":"outras edificações do tipo (uso comércio), com utilização múltipla",
    "outras edificações de uso de serviço, com utilização múltipla":"outras edificações do tipo (uso serviço), com utilização múltipla",
    "outras edificações de uso especial, com utilização múltipla":"outras edificações do tipo (uso especial), com utilização múltipla",
    "prédio com uso exclusivamente residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso exclusivamente residencial",
    "prédio com uso misto, predominância de uso não residencial, não em condomínio":"prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)",
    "prédio com uso misto, predominância de uso residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)",
    "prédio de escritório ou consultório, com ou sem lojas, não em condomínio":"prédio de escritório ou consultório, não em condomínio, com ou sem lojas",
    "flat de uso comercial (semelhante a hotel)":"flat, não residencial",
    "flat residencial em condomínio":"flat, residencial"
}

In [10]:
df_iptu = pd.read_csv('IPTU/IPTU_2020.zip',\
             compression='zip',\
             encoding='iso-8859-9',\
             sep=';',\
             decimal=',',\
             error_bad_lines=False)

In [12]:
# Obter o SQ (Setor e Quadra)
df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

# Calculando chave 'SQ'
df_iptu['sq'] = \
df_iptu['setor'] + df_iptu['quadra']

# Normalizando o lote condominial
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

# Individualizando a testada e pavimentos por lote (SQLC)
df_iptu['sqlc'] = df_iptu['setor'] + \
df_iptu['quadra'] + df_iptu['lote'] + \
df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

# Adequando variáveis à fração ideal
df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']

# Adequar os usos ao padrão estabelecido
df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
            .isin(usos),\
            'TIPO DE USO DO IMOVEL'] = \
df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
             .isin(usos)]['TIPO DE USO DO IMOVEL']\
.apply(lambda x: usos[x])

In [13]:
df_iptu[df_iptu['AREA CONSTRUIDA'] == 0].sqlc.nunique()

105031

In [14]:
df_iptu[(df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2) & (df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00')].sqlc.nunique()

26704

In [15]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2]['NUMERO DO CONTRIBUINTE'].nunique()

1856258

In [16]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2]['AREA CONSTRUIDA'].sum()

281495187

In [17]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2]['area_ocupada'].sum()

46610321.76059998

In [21]:
df_iptu['area_ocupada'].sum()

229194352.45139995

In [23]:
df_iptu[df_iptu.sqlc == '037033000010']['area_ocupada'].sum()

1247.6198

In [24]:
df_iptu[df_iptu.sqlc == '037033000010']['AREA CONSTRUIDA'].sum()

9950

In [28]:
27769/2.8


9917.5

In [ ]:
# setores_mae10 = ['037', '039']

In [29]:
df_iptu[df_iptu.setor == '037']['area_ocupada'].sum()

814376.4892000001

In [33]:
df_iptu[df_iptu.setor == '037']['AREA CONSTRUIDA'].sum()*3.4

12794070.799999999

In [37]:
df_iptu[['setor', 'AREA CONSTRUIDA', 'area_ocupada']].groupby('setor').sum().to_csv('resultados/areas_por_setor.csv')

In [18]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2]['area_terreno'].sum()

110653065.32809995

In [19]:
281495187/110653065

2.5439438753910704

In [20]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] <= 2].sqlc.nunique()

1583147

In [20]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] <= 2]['NUMERO DO CONTRIBUINTE'].nunique()

1642386

In [32]:
df_iptu[df_iptu['QUANTIDADE DE PAVIMENTOS'] > 2][['sqlc', 'AREA CONSTRUIDA']].groupby('sqlc').sum().sort_values(['AREA CONSTRUIDA'], ascending=True)

,AREA CONSTRUIDA
sqlc,
157219000002,14
060184003000,14
172151005500,20
163247008800,23
246032010200,24
...,...
141002000200,400000
141057001400,400000
147001000100,419167


In [34]:
df_iptu.loc[df_iptu.sqlc == '082517001400']

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,TIPO DE CONTRIBUINTE 1,CPF/CNPJ DO CONTRIBUINTE 1,NOME DO CONTRIBUINTE 1,TIPO DE CONTRIBUINTE 2,CPF/CNPJ DO CONTRIBUINTE 2,NOME DO CONTRIBUINTE 2,...,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,setor,quadra,sq,lote,sqlc,area_terreno,area_ocupada
1670808,0825170014-0,2020,1,11/01/20,PESSOA JURIDICA (CNPJ),XXXXXX3000XXXX,UNIVERSIDADE DE SAO PAULO USP,NaN,,NaN,...,1963,1,0,082,517,082517,0014,082517001400,4307493.0,320000.0


In [38]:
df_iptu.loc[df_iptu.sqlc == '299148000100']

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,TIPO DE CONTRIBUINTE 1,CPF/CNPJ DO CONTRIBUINTE 1,NOME DO CONTRIBUINTE 1,TIPO DE CONTRIBUINTE 2,CPF/CNPJ DO CONTRIBUINTE 2,NOME DO CONTRIBUINTE 2,...,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE,setor,quadra,sq,lote,sqlc,area_terreno,area_ocupada
3401830,2991480001-5,2020,1,11/01/20,PESSOA JURIDICA (CNPJ),XXXXXX1500XXXX,WTORRE SAO PAULO EMPREEND. IMOBILIARIOS S/A,NaN,,NaN,...,2008,1,0,299,148,299148,0001,299148000100,54082.0,45769.0


In [39]:
df_iptu.columns

Index(['NUMERO DO CONTRIBUINTE', 'ANO DO EXERCICIO', 'NUMERO DA NL',
       'DATA DO CADASTRAMENTO', 'TIPO DE CONTRIBUINTE 1',
       'CPF/CNPJ DO CONTRIBUINTE 1', 'NOME DO CONTRIBUINTE 1',
       'TIPO DE CONTRIBUINTE 2', 'CPF/CNPJ DO CONTRIBUINTE 2',
       'NOME DO CONTRIBUINTE 2', 'NUMERO DO CONDOMINIO', 'CODLOG DO IMOVEL',
       'NOME DE LOGRADOURO DO IMOVEL', 'NUMERO DO IMOVEL',
       'COMPLEMENTO DO IMOVEL', 'BAIRRO DO IMOVEL', 'REFERENCIA DO IMOVEL',
       'CEP DO IMOVEL', 'QUANTIDADE DE ESQUINAS/FRENTES', 'FRACAO IDEAL',
       'AREA DO TERRENO', 'AREA CONSTRUIDA', 'AREA OCUPADA',
       'VALOR DO M2 DO TERRENO', 'VALOR DO M2 DE CONSTRUCAO',
       'ANO DA CONSTRUCAO CORRIGIDO', 'QUANTIDADE DE PAVIMENTOS',
       'TESTADA PARA CALCULO', 'TIPO DE USO DO IMOVEL',
       'TIPO DE PADRAO DA CONSTRUCAO', 'TIPO DE TERRENO',
       'FATOR DE OBSOLESCENCIA', 'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
       'MES DE INICIO DA VIDA DO CONTRIBUINTE', 'FASE DO CONTRIBUINTE',
       'seto

## Carregando e pré-processando as quadras

In [5]:
# Carregando geometria das quadras fiscais
gdf_quadras = gpd.GeoDataFrame.from_file(f'zip://GIS/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF')

gdf_quadras['sq'] = gdf_quadras.qd_setor + gdf_quadras.qd_fiscal

gdf_quadras = gdf_quadras.loc[gdf_quadras.qd_tipo == 'F'][['sq', 'geometry']].dissolve(by='sq')

# gdf_quadras.geometry = gdf_quadras.representative_point()

## Agrupando os tipos de uso

In [6]:
df_iptu.columns

Index(['NUMERO DO CONTRIBUINTE', 'ANO DO EXERCICIO', 'NUMERO DA NL',
       'DATA DO CADASTRAMENTO', 'TIPO DE CONTRIBUINTE 1',
       'CPF/CNPJ DO CONTRIBUINTE 1', 'NOME DO CONTRIBUINTE 1',
       'TIPO DE CONTRIBUINTE 2', 'CPF/CNPJ DO CONTRIBUINTE 2',
       'NOME DO CONTRIBUINTE 2', 'NUMERO DO CONDOMINIO', 'CODLOG DO IMOVEL',
       'NOME DE LOGRADOURO DO IMOVEL', 'NUMERO DO IMOVEL',
       'COMPLEMENTO DO IMOVEL', 'BAIRRO DO IMOVEL', 'REFERENCIA DO IMOVEL',
       'CEP DO IMOVEL', 'QUANTIDADE DE ESQUINAS/FRENTES', 'FRACAO IDEAL',
       'AREA DO TERRENO', 'AREA CONSTRUIDA', 'AREA OCUPADA',
       'VALOR DO M2 DO TERRENO', 'VALOR DO M2 DE CONSTRUCAO',
       'ANO DA CONSTRUCAO CORRIGIDO', 'QUANTIDADE DE PAVIMENTOS',
       'TESTADA PARA CALCULO', 'TIPO DE USO DO IMOVEL',
       'TIPO DE PADRAO DA CONSTRUCAO', 'TIPO DE TERRENO',
       'FATOR DE OBSOLESCENCIA', 'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
       'MES DE INICIO DA VIDA DO CONTRIBUINTE', 'FASE DO CONTRIBUINTE',
       'seto

In [7]:
len(df_iptu)

3498644

In [8]:
df_iptu.loc[:, 'TIPO DE USO DO IMOVEL'] = df_iptu.loc[:, 'TIPO DE USO DO IMOVEL'].str.capitalize()

In [9]:
df_iptu.loc[:, 'TIPO DE USO DO IMOVEL'].value_counts()

Apartamento em condomínio                                                                                                                             1363685
Residência                                                                                                                                            1081450
Residência coletiva, exclusive cortiço (mais de uma residência no lote)                                                                                203816
Garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial                                                                 196802
Escritório/consultório em condomínio (unidade autônoma)                                                                                                138156
Terreno                                                                                                                                                109885
Residência e outro uso (predominância residencial)  

In [10]:
usos_agregados = {
    'Residencial':[
        'Apartamento em condomínio',
        'Apartamento',
        'Residência',
        'Garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial',
        'Garagem em edifício de uso exclusivamente residencial',
        'Residência e outro uso (predominância residencial)',
        'Flat residencial em condomínio',
        'Flat, residencial',
        'Prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)',
        'Prédio de apartamento, não em condomínio, de uso exclusivamente residencial',
        'Prédio com uso exclusivamente residencial, não em condomínio',
        'Prédio com uso misto, predominância de uso residencial, não em condomínio',
        'Cortiço',
        'Cortiço (habitação coletiva subnormal)',
        'Asilo, orfanato, creche, seminário ou convento',
        'Residência coletiva, exclusive cortiço (mais de uma residência no lote)',
        'Residência coletiva (mais de uma residência no lote), exclusive cortiço',
        'Hotel, pensão ou hospedaria'
    ],
    'Comercial e Serviços': [
        'Escritório/consultório em condomínio (unidade autônoma)',
        'Loja',
        'Loja e residência (predominância comercial)',
        'Loja em edifício em condomínio',
        'Escritório ou consultório',
        'Garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto',
        'Garagem em edifício de escritórios, consultórios ou misto',
        'Prédio de escritório ou consultório, com ou sem lojas, não em condomínio',
        'Prédio de escritório ou consultório, não em condomínio, com ou sem lojas',
        'Outras edificações do tipo (uso comércio), com utilização múltipla',
        'Outras edificações do tipo (uso serviço), com utilização múltipla',
        'Outras edificações do tipo (uso coletivo), com utilização múltipla',
        'Outras edificações do tipo (uso especial), com utilização múltipla',
        'Flat de uso comercial (semelhante a hotel)',
        'Flat, não residencial',
        'Loja em edifício em condomínio (unidade autônoma)',
        'Outras edificações de uso comercial, com utilização múltipla',
        'Garagem (unidade autônoma) de prédio de garagens',
        'Garagem, em prédio de garagens',
        'Estacionamento e garagem, não em condomínio',
        'Escola',
        'Templo',
        'Outras edificações de uso de serviço, com utilização múltipla',
        'Garagem (exclusive em prédio em condomínio)',
        'Posto de serviço',
        'Posto de serviço (combustíveis)',
        'Hospital, ambulatório, casa de saúde e assemelhados',
        'Outras edificações de uso especial, com utilização múltipla',
        'Cinema, teatro, casa de diversão, clube ou congênere',
        'Clube esportivo',
        'Estação radioemissora, de televisão ou empresa jornalística',
        'Prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)',
        'Prédio com uso misto, predominância de uso não residencial, não em condomínio',
        'Outras edificações de uso coletivo, com utilização múltipla'
    ],
    'Terrenos, indústria, galpões e outros':[
        'Terreno',
        'Indústria',
        'Oficina',
        'Armazéns gerais e depósitos'
    ]
}

In [11]:
usos_agregados['Residencial']

['Apartamento em condomínio',
 'Apartamento',
 'Residência',
 'Garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial',
 'Garagem em edifício de uso exclusivamente residencial',
 'Residência e outro uso (predominância residencial)',
 'Flat residencial em condomínio',
 'Flat, residencial',
 'Prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)',
 'Prédio de apartamento, não em condomínio, de uso exclusivamente residencial',
 'Prédio com uso exclusivamente residencial, não em condomínio',
 'Prédio com uso misto, predominância de uso residencial, não em condomínio',
 'Cortiço',
 'Cortiço (habitação coletiva subnormal)',
 'Asilo, orfanato, creche, seminário ou convento',
 'Residência coletiva, exclusive cortiço (mais de uma residência no lote)',
 'Residência coletiva (mais de uma residência no lote), exclusive cortiço',
 'Hotel, pensão ou hospedaria']

## Agregando por uso agregado e espacializando

In [12]:
usos_agregados.keys()

dict_keys(['Residencial', 'Comercial e Serviços', 'Terrenos, indústria, galpões e outros'])

In [13]:
for k in usos_agregados.keys():
    df_iptu\
    .loc[df_iptu['TIPO DE USO DO IMOVEL'].isin(usos_agregados[k]), \
        'uso_agregado'] = k

In [14]:
df_iptu.uso_agregado.value_counts()

Residencial                              2947359
Comercial e Serviços                      409471
Terrenos, indústria, galpões e outros     141814
Name: uso_agregado, dtype: int64

In [15]:
df_iptu.loc[df_iptu.uso_agregado.isna(), 'TIPO DE USO DO IMOVEL'].value_counts()

Series([], Name: TIPO DE USO DO IMOVEL, dtype: int64)

In [16]:
sq_area = df_iptu[['sq', \
                    'AREA CONSTRUIDA']]\
.groupby(['sq'])\
.agg({'AREA CONSTRUIDA': 'sum'})\
.rename(columns = {'AREA CONSTRUIDA': 'area_construida'})

In [17]:
iptu_agg = df_iptu[['sq', \
                    'AREA CONSTRUIDA',
                    'uso_agregado']]\
.groupby(['sq', 'uso_agregado'])\
.agg({'AREA CONSTRUIDA': 'sum'})\
.rename(columns = {'AREA CONSTRUIDA': 'area_construida'})

In [18]:
iptu_agg_qd = iptu_agg.reset_index().merge(sq_area, on='sq', how='left')

In [19]:
iptu_agg_qd = \
iptu_agg_qd.loc[(iptu_agg_qd.area_construida_y != 0) | \
               (iptu_agg_qd.area_construida_x != 0)]

In [20]:
iptu_agg_qd

,sq,uso_agregado,area_construida_x,area_construida_y
0,001003,Comercial e Serviços,22140,29926
1,001003,Residencial,7786,29926
2,001003,"Terrenos, indústria, galpões e outros",0,29926
3,001004,Comercial e Serviços,13305,20715
4,001004,Residencial,5994,20715
...,...,...,...,...
103915,310116,Comercial e Serviços,180,1313
103916,310116,Residencial,1133,1313
103917,310117,Comercial e Serviços,728,22601
103918,310117,Residencial,21873,22601


In [21]:
# Gerando GeoDataFrame
gdf_iptu_agg = gdf_quadras.merge(iptu_agg_qd, on='sq', how='right')

In [22]:
gdf_iptu_agg

,sq,geometry,uso_agregado,area_construida_x,area_construida_y
0,001003,"POLYGON ((333415.056 7396354.502, 333412.306 7...",Comercial e Serviços,22140,29926
1,001003,"POLYGON ((333415.056 7396354.502, 333412.306 7...",Residencial,7786,29926
2,001003,"POLYGON ((333415.056 7396354.502, 333412.306 7...","Terrenos, indústria, galpões e outros",0,29926
3,001004,"POLYGON ((333547.169 7396521.849, 333542.688 7...",Comercial e Serviços,13305,20715
4,001004,"POLYGON ((333547.169 7396521.849, 333542.688 7...",Residencial,5994,20715
...,...,...,...,...,...
103027,310116,"MULTIPOLYGON (((332734.442 7383503.772, 332721...",Comercial e Serviços,180,1313
103028,310116,"MULTIPOLYGON (((332734.442 7383503.772, 332721...",Residencial,1133,1313
103029,310117,"MULTIPOLYGON (((333231.482 7383452.112, 333232...",Comercial e Serviços,728,22601
103030,310117,"MULTIPOLYGON (((333231.482 7383452.112, 333232...",Residencial,21873,22601


In [23]:
pd.pivot_table(gdf_iptu_agg, values='area_construida_x', index=['sq'], columns=['uso_agregado'], aggfunc=np.sum)

uso_agregado,Comercial e Serviços,Residencial,"Terrenos, indústria, galpões e outros"
sq,,,
001003,22140.0,7786.0,0.0
001004,13305.0,5994.0,1416.0
001005,10246.0,5431.0,1586.0
001006,4279.0,3137.0,270.0
001007,12521.0,13246.0,300.0
...,...,...,...
310114,NaN,5025.0,NaN
310115,211.0,10549.0,244.0
310116,180.0,1133.0,NaN


In [24]:
gdf_iptu_pivot = \
gdf_quadras.merge(\
pd.pivot_table(gdf_iptu_agg, values='area_construida_x', index=['sq'], columns=['uso_agregado'], aggfunc=np.sum),\
                 on='sq', how='left')

In [25]:
gdf_iptu_pivot.replace(np.nan, 0, inplace=True)

In [26]:
gdf_iptu_pivot

,geometry,Comercial e Serviços,Residencial,"Terrenos, indústria, galpões e outros"
sq,,,,
001003,"POLYGON ((333415.056 7396354.502, 333412.306 7...",22140.0,7786.0,0.0
001004,"POLYGON ((333547.169 7396521.849, 333542.688 7...",13305.0,5994.0,1416.0
001005,"POLYGON ((333559.635 7396520.416, 333574.733 7...",10246.0,5431.0,1586.0
001006,"POLYGON ((333599.160 7396343.566, 333593.816 7...",4279.0,3137.0,270.0
001007,"POLYGON ((333689.258 7396305.230, 333688.459 7...",12521.0,13246.0,300.0
...,...,...,...,...
310116,"MULTIPOLYGON (((332734.442 7383503.772, 332721...",180.0,1133.0,0.0
310117,"MULTIPOLYGON (((333231.482 7383452.112, 333232...",728.0,21873.0,0.0
310118,"POLYGON ((332710.926 7384374.222, 332722.074 7...",0.0,25215.0,0.0


In [27]:
import colorsys

In [28]:
# colorsys.rgb_to_hsv(22140/100000, 25000/100000, 0.5)[0]

In [29]:
def calc_hue(x):
    at = x['Comercial e Serviços'] + x['Residencial'] + x['Terrenos, indústria, galpões e outros']
    if at == 0:
        rgb = (0, 0, 0)
    else:
        rgb = (x['Residencial']/at,\
               x['Comercial e Serviços']/at,\
               x['Terrenos, indústria, galpões e outros']/at)
    hue = colorsys.rgb_to_hsv(*rgb)[0]
    return hue

In [30]:
# totais_areas = gdf_iptu_pivot['Comercial e Serviços'] + gdf_iptu_pivot['Residencial'] + gdf_iptu_pivot['Terrenos, indústria, galpões e outros']

In [31]:
gdf_iptu_pivot['hue'] = gdf_iptu_pivot.apply(calc_hue, axis=1)

In [32]:
gdf_iptu_pivot.to_file('resultados/tipos_de_uso/seila.gpkg', driver='GPKG')

## Gera raster dos tipos de uso para cada ano

In [33]:
from pathlib import Path
import os

In [34]:
usos_df = []

In [35]:
for path in Path('IPTU').rglob('*.zip'):
    print(path.name)
    
    # 1. Carregar dados do arquivo
    # OBS.: O IPTU de 2016 está separado por ',' e não por ';'
    if path.name == "IPTU_2016.zip":
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=',',\
                     decimal=',',\
                     error_bad_lines=False)
    else:
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)
    
    # Obter o SQ (Setor e Quadra)
    df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
    df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

    # Calculando chave 'SQ'
    df_iptu['sq'] = \
    df_iptu['setor'] + df_iptu['quadra']

    # Normalizando o lote condominial
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

    # Individualizando a testada e pavimentos por lote (SQLC)
    df_iptu['sqlc'] = df_iptu['setor'] + \
    df_iptu['quadra'] + df_iptu['lote'] + \
    df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

    df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
    df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
    df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
    df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
    df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
    df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

    # Adequando variáveis à fração ideal
    df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
    df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']
    
    # Paadronizando tipo de uso com a primeira letra em maíusculo
    df_iptu.loc[:, 'TIPO DE USO DO IMOVEL'] = df_iptu.loc[:, 'TIPO DE USO DO IMOVEL'].str.capitalize()

    # Adequar os usos ao padrão estabelecido
    df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
                .isin(usos),\
                'TIPO DE USO DO IMOVEL'] = \
    df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
                 .isin(usos)]['TIPO DE USO DO IMOVEL']\
    .apply(lambda x: usos[x])
    
    for k in usos_agregados.keys():
        df_iptu\
        .loc[df_iptu['TIPO DE USO DO IMOVEL'].isin(usos_agregados[k]), \
            'uso_agregado'] = k
    
    # Auditando os tipos de uso
    print(df_iptu.loc[df_iptu.uso_agregado.isna(), 'TIPO DE USO DO IMOVEL'].value_counts())
    
    sq_area = df_iptu[['sq', \
                       'AREA CONSTRUIDA']]\
        .groupby(['sq'])\
        .agg({'AREA CONSTRUIDA': 'sum'})\
        .rename(columns = {'AREA CONSTRUIDA': 'area_construida'})
    
    iptu_agg = df_iptu[['sq', \
                        'AREA CONSTRUIDA',
                        'uso_agregado']]\
    .groupby(['sq', 'uso_agregado'])\
    .agg({'AREA CONSTRUIDA': 'sum'})\
    .rename(columns = {'AREA CONSTRUIDA': 'area_construida'})
    
    iptu_agg_qd = iptu_agg.reset_index().merge(sq_area, on='sq', how='left')
    
    iptu_agg_qd = \
    iptu_agg_qd.loc[(iptu_agg_qd.area_construida_y != 0) | \
                   (iptu_agg_qd.area_construida_x != 0)]
    
    df_iptu_pivot = \
    pd.pivot_table(iptu_agg_qd, values='area_construida_x', index=['sq'], columns=['uso_agregado'], aggfunc=np.sum)

    df_iptu_pivot.replace(np.nan, 0, inplace=True)
    
    df_iptu_pivot['hue'] = df_iptu_pivot.apply(calc_hue, axis=1)
    
    df_iptu_pivot.rename(columns={'Comercial e Serviços':f'{path.name[-8:-4]}_com',\
                                  'Residencial':f'{path.name[-8:-4]}_res',\
                                  'Terrenos, indústria, galpões e outros':f'{path.name[-8:-4]}_outros',\
                                  'hue':f'{path.name[-8:-4]}_hue'},\
                        inplace=True)
    
    usos_df.append(df_iptu_pivot)

IPTU_2010.zip


b'Skipping line 2445507: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Series([], Name: TIPO DE USO DO IMOVEL, dtype: int64)
IPTU_1996.zip


/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Residência vertical - padrão c    1
Name: TIPO DE USO DO IMOVEL, dtype: int64
IPTU_2007.zip


...

Series([], Name: TIPO DE USO DO IMOVEL, dtype: int64)
IPTU_2003.zip


b'Skipping line 2288886: expected 35 fields, saw 43\n'


Series([], Name: TIPO DE USO DO IMOVEL, dtype: int64)
IPTU_2012.zip


b'Skipping line 2526350: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Series([], Name: TIPO DE USO DO IMOVEL, dtype: int64)


In [36]:
usos_df

[uso_agregado  2010_com  2010_res  2010_outros  2010_hue
 sq                                                     
 001003         31919.0    7786.0          0.0  0.292678
 001004         12648.0    6274.0       1416.0  0.261248
 001005         10086.0    5431.0       1586.0  0.257941
 001006          4216.0    3277.0        270.0  0.206327
 001007         12926.0   11108.0        300.0  0.190665
 ...                ...       ...          ...       ...
 310114           180.0    4586.0          0.0  0.006542
 310115           167.0   10549.0        244.0  0.998764
 310116           180.0    1133.0          0.0  0.026478
 310117           728.0   21283.0          0.0  0.005701
 310118             0.0   25215.0          0.0  0.000000
 
 [41624 rows x 4 columns],
 uso_agregado  1996_com  1996_res  1996_outros  1996_hue
 sq                                                     
 001003         32538.0    6616.0          0.0  0.299445
 001004         13292.0    6984.0        594.0  0.249462
 0

In [37]:
# usos_df
df_usos = \
pd.concat(usos_df, axis=1).reindex(columns=sorted(pd.concat(usos_df, axis=1).columns))

In [38]:
df_usos.loc['036008']

uso_agregado
1995_com        5708.000000
1995_hue           0.013042
1995_outros        0.000000
1995_res       72942.000000
1996_com        6591.000000
                   ...     
2019_res       68482.000000
2020_com       34656.000000
2020_hue           0.084343
2020_outros        0.000000
2020_res       68482.000000
Name: 036008, Length: 104, dtype: float64

In [39]:
# Gerando GeoDataFrame
gdf_iptu_usos = gdf_quadras.merge(df_usos, on='sq', how='left')

In [40]:
gdf_iptu_usos.to_file('resultados/tipos_de_uso/IPTU_quadras_usos.gpkg', driver='GPKG')

In [41]:
# gdf_iptu_usos